In [11]:
import pandas as pd
import numpy as np
import datetime
import gc
import glob

In [12]:
file_list=glob.glob("/home/jian/BigLots/2017_Daily_Sales/"+"*.txt")
file_list=sorted(file_list)

In [13]:
file_list

['/home/jian/BigLots/2017_Daily_Sales/MediaStormDailySales01.txt',
 '/home/jian/BigLots/2017_Daily_Sales/MediaStormDailySales02.txt',
 '/home/jian/BigLots/2017_Daily_Sales/MediaStormDailySales03.txt',
 '/home/jian/BigLots/2017_Daily_Sales/MediaStormDailySales04.txt',
 '/home/jian/BigLots/2017_Daily_Sales/MediaStormDailySales05.txt',
 '/home/jian/BigLots/2017_Daily_Sales/MediaStormDailySales06.txt',
 '/home/jian/BigLots/2017_Daily_Sales/MediaStormDailySales07.txt',
 '/home/jian/BigLots/2017_Daily_Sales/MediaStormDailySales08.txt',
 '/home/jian/BigLots/2017_Daily_Sales/MediaStormDailySales09.txt',
 '/home/jian/BigLots/2017_Daily_Sales/MediaStormDailySales10.txt',
 '/home/jian/BigLots/2017_Daily_Sales/MediaStormDailySales11.txt',
 '/home/jian/BigLots/2017_Daily_Sales/MediaStormDailySales12.txt',
 '/home/jian/BigLots/2017_Daily_Sales/MediaStormDailySales13.txt']

In [16]:
all_daily_new_received_by_day=pd.DataFrame()
i=0
for file in file_list:
    df=pd.read_table(file,dtype=str,sep="|")
    df['transaction_dt']=df['transaction_dt'].apply(lambda x: datetime.datetime.strptime(x,"%Y-%m-%d").date())
    df['subclass_transaction_amt']=df['subclass_transaction_amt'].astype(float)
    df['subclass_transaction_units']=df['subclass_transaction_units'].astype(int)
    # all_daily_new_received=all_daily_new_received.append(df)
    i+=1
    print("File : "+str(i)+" | "+file+" | " + str(df['transaction_dt'].min())+" to "+str(df['transaction_dt'].max()))
    df['Rewards_Label']=np.where(pd.isnull(df['customer_id_hashed']),"Non_Rewards","Rewards")
    df_agg_sales=df.groupby(['location_id','transaction_dt','Rewards_Label'])['subclass_transaction_amt'].sum().to_frame().reset_index()
    
    df_agg_sales_total=df.groupby(['location_id','transaction_dt'])['subclass_transaction_amt'].sum().to_frame().reset_index()
    df_agg_sales_total=df_agg_sales_total.rename(columns={"subclass_transaction_amt":"total_sales"})
    
    df_agg_sales_rewards=df_agg_sales[df_agg_sales['Rewards_Label']=="Rewards"]
    del df_agg_sales_rewards['Rewards_Label']
    df_agg_sales_rewards=df_agg_sales_rewards.rename(columns={"subclass_transaction_amt":"rewards_sales"})
    
    df_agg_sales_Non_rewards=df_agg_sales[df_agg_sales['Rewards_Label']=="Non_Rewards"]
    del df_agg_sales_Non_rewards['Rewards_Label']
    df_agg_sales_Non_rewards=df_agg_sales_Non_rewards.rename(columns={"subclass_transaction_amt":"Non_rewards_sales"})
    
    df_app=pd.merge(df_agg_sales_total,df_agg_sales_rewards,on=['location_id','transaction_dt'],how="left")
    df_app=pd.merge(df_app,df_agg_sales_Non_rewards,on=['location_id','transaction_dt'],how="left")

all_daily_new_received_by_day=all_daily_new_received_by_day.append(df_app)

File : 1 | /home/jian/BigLots/2017_Daily_Sales/MediaStormDailySales01.txt | 2017-07-30 to 2017-08-05
File : 2 | /home/jian/BigLots/2017_Daily_Sales/MediaStormDailySales02.txt | 2017-08-06 to 2017-08-12
File : 3 | /home/jian/BigLots/2017_Daily_Sales/MediaStormDailySales03.txt | 2017-08-13 to 2017-08-19
File : 4 | /home/jian/BigLots/2017_Daily_Sales/MediaStormDailySales04.txt | 2017-08-20 to 2017-08-26
File : 5 | /home/jian/BigLots/2017_Daily_Sales/MediaStormDailySales05.txt | 2017-08-27 to 2017-09-02
File : 6 | /home/jian/BigLots/2017_Daily_Sales/MediaStormDailySales06.txt | 2017-09-03 to 2017-09-09
File : 7 | /home/jian/BigLots/2017_Daily_Sales/MediaStormDailySales07.txt | 2017-09-10 to 2017-09-16
File : 8 | /home/jian/BigLots/2017_Daily_Sales/MediaStormDailySales08.txt | 2017-09-17 to 2017-09-23
File : 9 | /home/jian/BigLots/2017_Daily_Sales/MediaStormDailySales09.txt | 2017-09-24 to 2017-09-30
File : 10 | /home/jian/BigLots/2017_Daily_Sales/MediaStormDailySales10.txt | 2017-10-01 to 

In [17]:
def add_week_end_dt(x):
    if x.weekday()==6:
        y=x+datetime.timedelta(days=6)
    else:
        y=x+datetime.timedelta(days=(5-x.weekday()))
    return y

all_daily_new_received_by_day['week_end_dt']=all_daily_new_received_by_day['transaction_dt'].apply(lambda x: add_week_end_dt(x))

In [18]:
all_daily_new_received_by_day.head(2)

,location_id,transaction_dt,total_sales,rewards_sales,Non_rewards_sales,week_end_dt
0,1,2017-10-22,12050.04,7158.29,4891.75,2017-10-28
1,1,2017-10-23,10155.06,5506.00,4649.06,2017-10-28


In [19]:
all_daily_new_received_by_week=all_daily_new_received_by_day.groupby(['location_id','week_end_dt'])['total_sales','rewards_sales','Non_rewards_sales'].sum().reset_index()
all_daily_new_received_by_week.head(2)

,location_id,week_end_dt,total_sales,rewards_sales,Non_rewards_sales
0,1,2017-10-28,72964.40,42682.01,30282.39
1,1001,2017-10-28,38806.63,8583.22,30223.41


In [20]:
import os
os.getcwd()

'/home/jian/Projects/Big_Lots/Analysis/2018_Q4/Read_data'

In [21]:
writer=pd.ExcelWriter("/home/jian/Projects/Big_Lots/Analysis/2018_Q4/Read_data/BL_2017_new_received_daily_data_summary_JL_20181203.xlsx",engine="xlsxwriter")
all_daily_new_received_by_day.to_excel(writer,"data_by_day",index=False)
all_daily_new_received_by_week.to_excel(writer,"data_by_week",index=False)
writer.save()